[video](https://www.youtube.com/watch?v=1AmLD1aY7cM)

# Thought, Action, Observation

Agent prompt
- output format instructions
- list of tools
- chat history
- examples

TODO explain how this works here

[create_react_agent docs](https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html)

In [33]:
from langchain import hub
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_react_agent
from langchain_ollama import ChatOllama
from langchain_community.tools import DuckDuckGoSearchRun

In [34]:
llm = ChatOllama(model='llama3.2')

In [35]:
prompt = hub.pull("hwchase17/react")
print(prompt.template)

C:\Python312\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [42]:
tools = load_tools(['llm-math'], llm=llm)
agent = create_react_agent(llm, 
                                    tools=tools, 
                                    prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [43]:
# Kinda hard to tell what's going on here, but yeah. The executor should take care of parsing the output, etc.
print(agent.get_graph().draw_ascii())

  +---------------------------------+  
  | Parallel<agent_scratchpad>Input |  
  +---------------------------------+  
             **         **             
           **             **           
          *                 *          
   +--------+          +-------------+ 
   | Lambda |          | Passthrough | 
   +--------+          +-------------+ 
             **         **             
               **     **               
                 *   *                 
 +----------------------------------+  
 | Parallel<agent_scratchpad>Output |  
 +----------------------------------+  
                   *                   
                   *                   
                   *                   
          +----------------+           
          | PromptTemplate |           
          +----------------+           
                   *                   
                   *                   
                   *                   
            +------------+             


In [47]:
# this works, but the LLM is pretty bad at following instructions so this often errors.
# agent_executor.invoke({"input": "what is 365842068 + 3409568092?"})

We should probably figure out how AgentExecutor works, and what format we need the chain to be in to work with the AgentExecutor, but for now, let's move on to LangGraph

# LangGraph

let's recreate a ReAct agent that can search the web and do math.

- [MessagesPlaceholder](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html)

In [163]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages.system import SystemMessage
from langchain_core.output_parsers import StrOutputParser

# state
from typing import List, Optional
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage

# misc
import re
import json

In [57]:
# For searching the web/doing calculations this should be good.
class Prediction(TypedDict):
    action: str
    args: Optional[str]

class AgentState(TypedDict):
    input: str # user request
    prediction: Prediction # agent's output
    scratchpad: List[BaseMessage]
    observation: str # most recent response from a tool

In [64]:
ddg_search = DuckDuckGoSearchRun()

def search(state: AgentState):
    args = state['prediction']['args']
    if args is None:
        return "No arguments found for search tool. Please provide a search query!"
    return ddg_search.invoke(args)

def eval(state: AgentState):
    # We'll only evaluate basic math here
    args = state['prediction']['args']
    if args is None:
        return "No arguments found for math tool. Please provide a numerical expression!"
    args = re.sub(r'[^0-9+-/*()]', '', args) # remove non-math related things
    try:
        result = eval(args)
        return f"{args} = {result}"
    except SyntaxError:
        return f"Invalid math expression: {args}"

In [207]:
def parse(text: str):
    text = text.replace('{{', '{')
    text = text.replace('}}', '}')
    try:
        dict_in_text = re.search(r'\{([^{}]*)\}', text).group(1)
        prediction = json.loads('{' + dict_in_text + '}')
        if 'action' in prediction and 'args' in prediction:
            return {'action': prediction['action'].strip(), 'args': prediction['args'].strip()}
        else:
            raise SyntaxError
    except:
        return {'action': 'retry', 'args': 'could not parse LLM output as JSON'}

def update_scratchpad(state: AgentState):
    # scratchpad will just contain one message here
    old = state.get('scratchpad')
    if old:
        txt = old[0].content
        last_line = txt.rsplit("\n", 1)[-1]
        step = int(re.match(r"\d+", last_line).group()) + 1
    else:
        txt = "Previous action observations:\n"
        step = 1
    txt += f"\n{step}. {state['observation']}"

    return {**state, "scratchpad": [SystemMessage(content=txt)]}

def get_llm_chain():
    llm = ChatOllama(model="llama3.2", max_tokens=4096)
    prompt_str = """
    Answer the following questions as best you can. Provide some reasoning, and then choose one of the following actions:

    1. Evaluate a math expression
    2. Search the web
    3. Respond with final answer, once previous action observations contain sufficient info to answer the question.

    Correspondingly, Action should be returned as a JSON string, following these formats:
    - {{ "action": "eval", "args": "NUMERICAL EXPRESSION" }}
    - {{ "action": "search", "args": "SEARCH QUERY" }}
    - {{ "action": "answer", "args": "ANSWER" }}
    
    Key Guidelines You MUST follow:

    Execute only one action per iteration.
    Keys and values in the Action JSON MUST be strings
    
    Your reply should strictly follow the format:

    Thought: Your brief thoughts (briefly summarize the info that will help ANSWER)
    Action: JSON formatted action
    Then the User will provide:
    Observation: Result of the action
    """
    prompt = ChatPromptTemplate(messages=[SystemMessage(content=prompt_str), 
                                          MessagesPlaceholder('scratchpad'),
                                          HumanMessagePromptTemplate.from_template("Question: {input}")], input_variables=['input'])
    # Assign str_output for debugging purposes
    agent = (RunnablePassthrough.assign(str_output=prompt | llm | StrOutputParser())
             | RunnablePassthrough.assign(prediction=lambda state: parse(state['str_output'])))
    return agent, prompt

In [213]:
chain, prompt = get_llm_chain()

# invoking the chain with no previous observations
print("Invoking chain with no previous observations")
result = chain.invoke({"input": "what is kanye west's net worth?", "scratchpad": []})
print(f"\nRAW TEXT:\n{result['str_output']}")
print(f"\nACTION:\n{result['prediction']}")

# invoking the chain with artificial previous observations
print("\n#############################\n")
print("Invoking chain with artificial previous observations")
result = chain.invoke({"input": "what is kanye west's net worth?", 
              "scratchpad": [SystemMessage(content='Previous action observations:\nKanye west has a net worth of 2 million dollars')]})
print(f"\nRAW TEXT:\n{result['str_output']}")
print(f"\nACTION:\n{result['prediction']}")

Invoking chain with no previous observations

RAW TEXT:
Thought: Searching for recent Kanye West net worth data to ensure accuracy.

Action: {{ "action": "search", "args": "Kanye West net worth 2024" }}

ACTION:
{'action': 'search', 'args': 'Kanye West net worth 2024'}

#############################

Invoking chain with artificial previous observations

RAW TEXT:
Thought: The observation contains enough information about Kanye West's net worth, so we can directly provide an answer.

Action: {{ "action": "answer", "args": "2 million dollars" }}

ACTION:
{'action': 'answer', 'args': '2 million dollars'}


{'input': "what is kanye west's net worth?",
 'scratchpad': [SystemMessage(content='Previous action observations:\nKanye west has a net worth of 2 million dollars', additional_kwargs={}, response_metadata={})],
 'str_output': 'Thought: Based on previous observation, Kanye West\'s net worth should be consistent with the given value.\n\nAction: {{ "action": "answer", "args": "2,000,000" }}\n\nPlease provide the result of this action.',
 'prediction': {'action': 'answer', 'args': '2,000,000'}}